In [ ]:
#importing necessary libraries
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf

In [ ]:
def load_data(data_path):
  dataset = pd.read_csv(data_path, header = 0, sep = ' ',
            names = ['x','y','dev1/dev2','orient','tx_pwr','col','rssi_1','rssi_2','rssi_3','rssi_4','rssi_5'])
  dataset.set_index(['x','y'])
  return dataset

In [ ]:
data_set = load_data("/content/omni_variable_txpower.txt")

In [ ]:
grouped = data_set.groupby(['x','y'])

In [ ]:
locations = set()
for i in zip(data_set['x'],data_set['y']):
  locations.add(i)

In [ ]:
data_sets = []
for k in list(locations)[:2]:
  data_sets.append(grouped.get_group(k))

In [ ]:
grouped_data = data_sets[0].groupby('tx_pwr')
tx_pwr = range(10,21)
data_sets_tx = []
for k in tx_pwr:
  data_sets_tx.append(grouped_data.get_group(k))

In [ ]:
df_input = [data_sets_tx[i][['tx_pwr','rssi_1']] for i in range(len(data_sets_tx))]

In [ ]:
#scaling the dataset
def scale_dataset(df_ip):
  scaler = MinMaxScaler()
  return [scaler, scaler.fit_transform(df_ip)]
scaler_set = []
data_scaled = []
for df in df_input:
  scaler, data = scale_dataset(df)
  scaler_set.append(scaler)
  data_scaled.append(data)

In [ ]:
def get_features(scaled_data):
  features = scaled_data[:,1]
  target = scaled_data[:,1]
  return [features, target]

In [ ]:
feature_target_set = []
for data in data_scaled:
  feature_target_set.append(get_features(data))

In [ ]:
def split_dataset(features, target):
  x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state=123, shuffle=False)
  x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state=123, shuffle=False)
  return [x_train, y_train, x_test, y_test, x_val, y_val]

In [ ]:
window_length = 5
batch_size = 5
num_features = 1

In [ ]:
train_set = []
test_set = []
validation_set = []
for data in feature_target_set:
  result = split_dataset(data[0], data[1])
  train_set.append([result[0],result[1]])
  test_set.append([result[2],result[3]])
  validation_set.append([result[4],result[5]])

In [ ]:
def generator(x,y):
  return TimeseriesGenerator(x,y, length = window_length, sampling_rate=1, batch_size=batch_size)

In [ ]:
test_generator = []
for test in test_set:
  test_generator.append(generator(test[0],test[1]))

In [ ]:
def make_prediction(model, test_gen):
  return model.predict_generator(test_gen)

In [ ]:
import pickle
models = []
for i in range(len(tx_pwr)):
  file_name = f'saved_model_{tx_pwr[i]}.pkl'
  models.append(pickle.load(open(file_name,'rb')))

In [ ]:
predictions = []
for i in range(len(test_generator)):
  predictions.append(make_prediction(models[i], test_generator[i]))

In [ ]:
def actual_pred(predictions, x_test, df_input, scaler):
  df_pred = pd.concat([pd.DataFrame(x_test[:][window_length:]),pd.DataFrame(predictions)],axis=1)
  reverse_trans = scaler.inverse_transform(df_pred)
  df_final = df_input[-predictions.shape[0]:]
  df_final['pred'] = reverse_trans[:,-1]
  return df_final

In [ ]:
final_preds = []
for i in range(len(tx_pwr)):
  df_final = actual_pred(predictions[i], test_set[i][0], df_input[i], scaler_set[i])
  final_preds.append(df_final['pred'])

In [ ]:
rssi_values = []
for preds in final_preds:
  rssi_values.append(list(preds)[-1])
rssi_values

In [ ]:
# ========================== SIMULATION PARAMETERS ====================================#
import numpy as np

N_DEVICES = 50
MAX_TX_POWER = 20 #dBm
SPLITTING_FACTOR = 0.5
TRANSMISSION_RANGE = 10 #meters
CHARGING_RANGE = 10 #meters
GAMMA = 3 #dB
SIGMA = -90 #dBm
ANTENNA_NUM = 1
GATEWAY_ANTENNA = 1
REF_DIST = 1 #meters
ALPHA = -20 #dB
BETA = 2
GATEWAY_POS = np.array((0,0)) # meters
DEPLOY_AREA = (45,45) # sq.meters
d = 1
SPREADING_FACTOR = 7
BANDWIDTH = 250 #KHz
HARV_EFF = 0.85 #Assuming linear conversion efficiency
NUM_SYMBOLS = 12
TIME_ON_AIR = NUM_SYMBOLS * ((2**SPREADING_FACTOR) / BANDWIDTH)
TX_POWER = range(10, 21)



In [ ]:
# Sample IoT devices uniformly around the fixed gateway
import numpy as np
import matplotlib.pyplot as plt
def deployment(n_devices, deploy_area):
  positions = np.random.uniform(-deploy_area,deploy_area, size = (n_devices,2))
  return positions

In [ ]:
# Plot the deployed IoT devices and gateway
positions = deployment(N_DEVICES, (DEPLOY_AREA[0])/2)
plt.rcParams["figure.figsize"] = [5, 5]
plt.plot(GATEWAY_POS[0],GATEWAY_POS[1],marker = "o", markerfacecolor="red")
plt.scatter(positions[:,0], positions[:,1])

In [ ]:
#Generating list of edges
import numpy as np
def generate_edgelist(n_devices, positions, gateway_pos):
  edge = []
  for i in range(0, n_devices+1):
    for j in range(i+1, n_devices+1):
      dist = 0
      if i == n_devices or j == n_devices:
        dist = np.linalg.norm(positions[i]-GATEWAY_POS)
      else:
        dist = np.linalg.norm(positions[i]-positions[j])
      if(dist < CHARGING_RANGE):
        edge.append((i,j))
        edge.append((j,i))
  return edge


In [ ]:
#construct adjacency list
def generate_adj_list(edges):
  adj_list = {}
  for edge in edges:
    v1 = edge[0]
    v2 = edge[1]
    if v1 in adj_list:
      adj_list[v1].add(v2)
    else:
      adj_list[v1] = {v2};
    if v2 in adj_list:
      adj_list[v2].add(v1)
    else:
      adj_list[v2] = {v1}
  return adj_list

In [ ]:
edges = generate_edgelist(N_DEVICES, positions, GATEWAY_POS)
edge_to_idx = {}
for i in range(len(edges)):
  edge_to_idx[edges[i]] = i

adj_list = generate_adj_list(edges)
for k in adj_list:
  print("{} :-> {}\n".format(k, adj_list[k]))


In [ ]:
#Check if the connected graph is connected or not
visited = [False for i in range(N_DEVICES+1)]
visited[N_DEVICES] = True
def dfs(src):
  for i in adj_list[src]:
    if not visited[i]:
      visited[i] = True
      dfs(i)

dfs(N_DEVICES)
isConnected = True
for i in visited:
  isConnected = isConnected and i

if isConnected:
  print("connected")
else:
  print("Not connected")

In [ ]:
#find how many devices each edge can charge
edge_charge = {}
for edge in edges:
  if N_DEVICES in adj_list[edge[0]]:
    edge_charge[edge] = len(adj_list[edge[0]])-1
  else:
    edge_charge[edge] = len(adj_list[edge[0]])


In [ ]:
#sort the edges in descending order depending upon the number of devices it charges
sorted_list = sorted(edge_charge.items(), key = lambda kv: kv[1], reverse = True)
sorted_edges = [e[0] for e in sorted_list]

In [ ]:
#Ensure all communications are half-duplex
#Function returns FALSE if neight router i nor j exists in the transmission set Sz
def HalfDuplex(edge, S_z):
  for e in S_z:
    if edge[0] in e or edge[1] in e:
      return True
  return False

In [ ]:
def get_edge_idx(edge):
  for i in range(len(edges)):
    if edges[i] == edge:
      return i


In [ ]:
#construct channel gain matrix
channel_gain = []
for u_pair in edges:
  gain = []
  for a1 in range(ANTENNA_NUM):
    row = []
    for a2 in range(ANTENNA_NUM):
      if u_pair[0] == N_DEVICES:
        dist = 1/(np.linalg.norm(GATEWAY_POS-positions[u_pair[1]]))**(2.5)
      elif u_pair[1] == N_DEVICES:
        dist = 1/(np.linalg.norm(positions[u_pair[0]]-GATEWAY_POS))**(2.5)
      else:
        dist = 1/(np.linalg.norm(positions[u_pair[0]]-positions[u_pair[1]]))**(2.5)
      row.append(dist)
    gain.append(row)
  channel_gain.append(gain)
channel_gain

In [ ]:
#construct channel gain matrix
from numpy.random import exponential
import random

#use Reyleigh channel fading
channel_gain = []
random.seed(0)
for u_pair in edges:
  gain = []
  if u_pair[0] == N_DEVICES:
    x = exponential(scale = 1, size = GATEWAY_ANTENNA*ANTENNA_NUM)
    next = 0
    for a1 in range(GATEWAY_ANTENNA):
      row = []
      for a2 in range(ANTENNA_NUM):
        dist = np.linalg.norm(GATEWAY_POS-positions[u_pair[1]])
        row.append(x[next]*ALPHA*(dist/REF_DIST)**(-BETA))
        next += 1
      gain.append(row)
  elif u_pair[1] == N_DEVICES:
    x = exponential(scale = 1, size = ANTENNA_NUM*GATEWAY_ANTENNA)
    next = 0
    for a1 in range(ANTENNA_NUM):
      row = []
      for a2 in range(GATEWAY_ANTENNA):
        dist = np.linalg.norm(positions[u_pair[0]]-GATEWAY_POS)
        row.append(x[next]*ALPHA*(dist/REF_DIST)**(-BETA))
        next += 1
      gain.append(row)
  else:
    x = exponential(scale = 1, size = ANTENNA_NUM**2)
    next = 0
    for a1 in range(ANTENNA_NUM):
      row = []
      for a2 in range(ANTENNA_NUM):
        dist = np.linalg.norm(positions[u_pair[0]]-positions[u_pair[1]])
        row.append(x[next]*ALPHA*(dist/REF_DIST)**(-BETA))
        next += 1
      gain.append(row)
  channel_gain.append(gain)


In [ ]:
!pip install gurobipy>=9.5.1

In [ ]:
from gurobipy import *
e = Env(empty = True)
e.setParam('WLSACCESSID', '')
e.setParam('WLSSECRET','')
e.setParam('LICENSEID',)
e.start()

In [ ]:
def mul(li,scalar):
    return [(i,scalar) for i in li]

In [ ]:
import math
def P_Allocation(S_z, gamma,max_pwr):

  #setting the problem variable
  prob = Model(env = e)
  prob.setParam("BarHomogeneous", 1)

  #setting the decision variables
  trans_set = set()
  recv_set = set()
  for links in S_z:
    trans_set.add(links[0])
    recv_set.add(links[1])
  
  trans_pwr = {(i,k): prob.addVar(lb = 0, ub = 1, vtype = GRB.BINARY, name="p_{0}_{1}".format(i,k)) for i in range(N_DEVICES+1) for k in range(len(tx_pwr))}

  eh_rate = {i: prob.addVar(lb = -GRB.INFINITY, vtype = GRB.CONTINUOUS, name = "delta_{0}".format(i)) for i in range(N_DEVICES)}

  recv_pwr = {n: prob.addVar(lb = -GRB.INFINITY, vtype = GRB.CONTINUOUS, name = "P_{0}".format(n)) for n in range(N_DEVICES)}

  denom = {i: prob.addVar(lb = -GRB.INFINITY, name = 'denom_{0}'.format(i)) for i in range(len(S_z))}

  #Define the objective function
  obj = sum(eh_rate[i] for i in range(N_DEVICES))

  prob.ModelSense = GRB.MAXIMIZE
  prob.setParam('NonConvex',2)

  prob.setObjective(obj)

  #Defining constraints

  #exactly one transmit level should be activated for a transmitting node
  constr_1 = {i: prob.addConstr(sum(trans_pwr[i,k] for k in range(max_pwr)) == 1, name = "constr_1_{0}".format(i)) 
          for i in trans_set}

  #Adding SINR_CONSTRAINT based on Rayleigh fading channels
  link_no = 0
  for links in S_z:
    i = links[0]
    j = links[1]

    temp = LinExpr(rssi_values[0], trans_pwr[i,0])
    for k in range(1, max_pwr):
      temp.add(LinExpr(rssi_values[k], trans_pwr[i,k]))
    
    temp2 = LinExpr()
    for l_rs in S_z:
      if l_rs != links:
        r = l_rs[0]
        s = l_rs[1]
        if j in adj_list[r]:
          for k in range(max_pwr):
            temp2.add(LinExpr(rssi_values[k], trans_pwr[r,k]))
    temp2.add(LinExpr(SIGMA))
    prob.addConstr(temp2*denom[link_no] == 1, name = 'constr_2_1_{0}'.format(link_no))
    prob.addConstr(temp*denom[link_no] >= gamma, name = 'constr_2_2_{0}'.format(link_no))
    link_no += 1

    #constraint for receieved power
    constr_3 = {n: prob.addConstr(recv_pwr[n] == sum(trans_pwr[i,k]*rssi_values[k] for k in range(max_pwr) for i in adj_list[n]), name = 'constr_3_{0}'.format(n)) for n in range(N_DEVICES)}

    #transmit power of not transmitting devices is 0
    constr_4 = {i: prob.addConstr(sum(trans_pwr[i,k] for k in range(max_pwr)) == 0, name = "constr_4_{0}".format(i)) 
            for i in range(N_DEVICES+1) if i not in trans_set}

    #constraint for energy harvesting rate
    constr_5 = {n: prob.addConstr(eh_rate[n] == recv_pwr[n]) for n in range(N_DEVICES)}

    #solve the problem
    # prob.computeIIS()
    # prob.write("model.ilp")
    # prob.feasRelaxS(2,True,False,True)
    # prob.feasRelaxS(1, False, False, True)
    prob.optimize()
    prob.write("out.lp")

    #get the value of decision variables
    antenna_weights = {}
    EH_rates = []
    flag = False
    if prob.status in (GRB.OPTIMAL, GRB.SUBOPTIMAL):
      flag = True
      for i in trans_set:
        antenna_weights[i] = []
        for k in range(max_pwr):
          if prob.getVarByName("p_{0}_{1}".format(i,k)).x == 1:
            antenna_weights[i].append(tx_pwr[k])
      for rate in ["delta_{0}".format(n) for n in range(N_DEVICES)]:
        EH_rates.append(HARV_EFF*(prob.getVarByName(rate).x))
    prob.reset(1)
    return [flag, antenna_weights, EH_rates]


In [ ]:
sched_len = []
for i in range(len(tx_pwr)):
  temp_edges = sorted_edges.copy()  
  z = 0
  #phase-1
  GAMMA = -10 #dBm
  antenna_wt = []
  H = []
  S = []
  A = []
  while len(sorted_edges) != 0:
    antenna_wt.append([])
    H.append([])
    S.append([])
    for edge in sorted_edges:
      if HalfDuplex(edge, S[z]) == False:
        tempS_z = S[z] + [edge]
        p_alloc_res = P_Allocation(tempS_z, GAMMA, i)
        if p_alloc_res[0]:
          S[z] = tempS_z
          # print(p_alloc_res)
          antenna_wt[z] = p_alloc_res[1]
          H[z] = p_alloc_res[2]
    if len(S[z]) == 0:
      break;
    for link in S[z]:
      sorted_edges.remove(link)
    z += 1
  print(antenna_wt)
  sched_len.append(len(S))
  sorted_edges = temp_edges.copy()